In [1]:
import numpy as np
import random
import pandas as pd
import math

In [2]:
import os
project_path = os.path.abspath(os.path.join(os.path.dirname(os.path.abspath('.')), '../'))
import sys
sys.path.append(project_path)
from util.visualization import draw_lines
from util.visualization import draw_scatters
from util.evaluate_process import mse, rmse, mae

In [3]:
from sklearn.model_selection import train_test_split

In [57]:
mseret = {}
scoreret = {}

In [18]:
df = pd.read_csv('linear_model_data.10d.v1.csv')

In [19]:
df.head()

,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,y
0,36.81,25.09,47.98,33.46,-5.32,27.67,98.92,45.85,68.12,139.28,705.905389
1,-48.88,12.36,-24.02,0.42,26.45,25.71,14.52,42.39,45.33,87.36,698.050785
2,-28.59,3.34,35.34,39.56,51.35,13.36,20.59,80.20,65.51,102.63,-57.348093
3,16.37,-10.79,4.47,56.09,34.14,72.48,41.76,82.41,45.08,64.42,136.479240
4,-18.86,-28.85,27.55,58.26,19.57,26.85,50.16,110.60,71.63,98.36,-199.452840


In [20]:
### LARs  like Stagewise

In [21]:
## 归一化
cf = df.copy()
mins = {k: min(df[k]) for k in df.columns}
maxs = {k: max(df[k]) for k in df.columns}
widths = {k: maxs[k] -mins[k] for k in df.columns}
###
for k in cf.columns:
    cf[k] = cf[k].map(lambda x: (x-mins[k])/widths[k])
cf.head()

,d0,d1,d2,d3,d4,d5,d6,d7,d8,d9,y
0,0.868495,0.650616,0.781523,0.533774,0.046328,0.276758,0.889990,0.258394,0.381731,0.995786,0.675993
1,0.010909,0.523175,0.059284,0.202646,0.364219,0.257097,0.045145,0.223715,0.153214,0.474867,0.673626
2,0.213971,0.432876,0.654730,0.594909,0.613368,0.133213,0.105906,0.602686,0.355560,0.628073,0.446011
3,0.663931,0.291421,0.345070,0.760573,0.441165,0.726251,0.317818,0.624837,0.150707,0.244708,0.504415
4,0.311349,0.110622,0.576587,0.782321,0.295377,0.268532,0.401902,0.907387,0.416926,0.585231,0.403193


In [22]:
feats = list(cf.columns)[:-1]
feats

['d0', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9']

In [23]:
Xtrain, Xtest, ytrain, ytest = train_test_split(cf[feats], cf.y)
len(Xtrain), len(Xtest)

(750, 250)

In [60]:
from sklearn import linear_model
clf = linear_model.Lars(n_nonzero_coefs=1)



clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret['lars'] = rst_mse
scoreret['lars'] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

0.02940536480855946 0.17147992538066797 0.1382357569206642
0.04416498154511983 0.04416498154511983


### huber regression

In [63]:
from sklearn.linear_model import HuberRegressor

clf = HuberRegressor()
model_name = 'huber'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

9.156077313393423e-05 0.00956873937015395 0.0033712955059952677
0.9974193291166068 0.9974193291166068


### bayes ard

In [64]:
from sklearn import linear_model
clf = linear_model.ARDRegression()

model_name = 'bayes_ard'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

8.92460467448665e-05 0.009447012583079716 0.0038295424030686752
0.9975339377718496 0.9975339377718496


### bayes ridge

In [65]:
from sklearn import linear_model
clf = linear_model.BayesianRidge()


model_name = 'bayes_ridge'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

8.922227166769083e-05 0.009445754160875184 0.0038285082055194443
0.9975339630259675 0.9975339630259675


In [66]:
from sklearn.linear_model import ElasticNet
clf = ElasticNet(random_state=0, l1_ratio=0, max_iter=9000)

model_name = 'ElasticNet'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

0.026425240921314555 0.16255842310170998 0.1312439056047523
0.14279817930567074 0.14279817930567074


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.462005512618799, tolerance: 0.002044025402747136



### Lasso
- 因为每个特征都有用, 所以应该用l2 , 不应该改用l1

In [67]:

from sklearn.linear_model import Lasso


clf = Lasso(alpha=0.00001)

model_name = 'lasso'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

8.892936648820013e-05 0.009430236820366715 0.0038164743713862102
0.9975335172649862 0.9975335172649862


In [68]:
from sklearn.linear_model import LassoCV
clf = LassoCV(cv=5, random_state=0)


model_name = 'lassoCV'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

8.852551398702259e-05 0.009408799816502772 0.0038011245995383976
0.997531319324601 0.997531319324601


In [69]:
from sklearn.linear_model import RANSACRegressor
from sklearn.datasets import make_regression
X, y = make_regression(
    n_samples=200, n_features=2, noise=4.0, random_state=0)
clf = RANSACRegressor(random_state=0)


model_name = 'ransac'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

8.924175244724817e-05 0.009446785296980564 0.0038292131480618242
0.9975339641502755 0.9975339641502755


In [70]:
from sklearn.linear_model import Ridge

clf = Ridge(alpha=1.0)

model_name = 'ridge'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

8.830040119033278e-05 0.009396829315802898 0.004478360961588505
0.9972609118522727 0.9972609118522727


In [71]:
from sklearn.linear_model import TheilSenRegressor
from sklearn.datasets import make_regression
X, y = make_regression(
    n_samples=200, n_features=2, noise=4.0, random_state=0)
clf = TheilSenRegressor(random_state=0)


model_name = 'TheilSenRegressor'
clf.fit(Xtrain, ytrain)

pred = clf.predict(Xtest)
rst_mse = mse(ytest, pred)
rst_score = clf.score(Xtrain, ytrain)
mseret[model_name] = rst_mse
scoreret[model_name] = rst_score
print(rst_mse, rmse(ytest, pred), mae(ytest, pred))
print(clf.score(Xtrain, ytrain), rst_score)
edf = pd.DataFrame({'target': ytest, 'pred': pred})
edf.sort_values(by='target', inplace=True)
draw_lines([i for i in range(len(ytest))], [edf.target, edf.pred], ['true', 'pred'])

9.106655155765638e-05 0.00954287962606971 0.0033952972387346694
0.9974400263929448 0.9974400263929448


In [72]:
mseret

{'lars': 0.02940536480855946,
 'huber': 9.156077313393423e-05,
 'bayes_ard': 8.92460467448665e-05,
 'bayes_ridge': 8.922227166769083e-05,
 'ElasticNet': 0.026425240921314555,
 'lasso': 8.892936648820013e-05,
 'lassoCV': 8.852551398702259e-05,
 'ransac': 8.924175244724817e-05,
 'ridge': 8.830040119033278e-05,
 'TheilSenRegressor': 9.106655155765638e-05}